# Leeds EDA

This notebook contains the EDA process of the Leeds City Council EV charging dataset, ranging from 2014 to 2021 Q1.  

Contents:  
[Combine Datasets](#Combine-Datasets)  

In [1]:
# Import the necessary packages
import os
import re
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Combine Datasets

The datasets are first combined before being analyised.  
----
But wait!  
Some of the files are in dd/mm/yyyy format.  
While the other files are in mm/dd/yyy format.  
We need to make sure all the files are in the same datetime format.  
The selected final format is dd/mm/yyyy.  
We have manually (and painstakingly) renamed the files.  
If they start with `MD` then the datetime format is mm/dd/yyyy.   
Else it is `DM` for dd/mm/yyyy.  

In [2]:
# Read in all .csv files in data
path = "../data/"
skip = ["combined_ev_charging_data.csv", "combined_ev_data.csv"]
os.chdir(path)
csv_files = csv_files = [f for f in os.listdir() if f.endswith('.csv') if f not in skip]
print(csv_files)
# Split the files into 2 groups according to their datetime format.  
md_files = [filename for filename in csv_files if filename.startswith('MD')]
dm_files = [filename for filename in csv_files if filename.startswith('DM')]

print("MD Filenames:", md_files)
print("DM Filenames:", dm_files)

['DM_2018 EVCP use Q3 and Q4.csv', 'DM_2017 EVCP use Q3 and Q4.csv', 'MD_2016 EVCP use Q3 and Q4.csv', 'DM_2021 EVCP use Q1.csv', 'DM_2014 EVCP use Q2.csv', 'MD_2019 EVCP use Q3 and Q4.csv', 'DM_2020 EVCP use Q3.csv', 'DM_2015 EVCP use Q3 and Q4.csv', 'MD_2014 EVCP use Q3 and Q4.csv', 'DM_2015 EVCP use Q1 and Q2.csv', 'DM_2016 EVCP use Q3 and Q4.csv', 'DM_2020 EVCP use Q1 and Q2.csv', 'DM_2019 EVCP use Q3 and Q4.csv', 'MD_2014 EVCP use Q2.csv', 'DM_2019 EVCP use Q1 and Q2.csv', 'MD_2015 EVCP use Q1 and Q2.csv', 'DM_2018 EVCP use Q1 and Q2.csv', 'DM_2016 EVCP use Q1 and Q2.csv', 'MD_2015 EVCP use Q3 and Q4.csv', 'DM_2014 EVCP use Q3 and Q4.csv', 'DM_2017 EVCP use Q1 and Q2.csv']
MD Filenames: ['MD_2016 EVCP use Q3 and Q4.csv', 'MD_2019 EVCP use Q3 and Q4.csv', 'MD_2014 EVCP use Q3 and Q4.csv', 'MD_2014 EVCP use Q2.csv', 'MD_2015 EVCP use Q1 and Q2.csv', 'MD_2015 EVCP use Q3 and Q4.csv']
DM Filenames: ['DM_2018 EVCP use Q3 and Q4.csv', 'DM_2017 EVCP use Q3 and Q4.csv', 'DM_2021 EVCP use 

Now, we can have a function that changes the datetime format.   

In [3]:
# Define a function to reformat datetime of multiple colmuns into selected format.
def convert_date_formats(dataframe, columns_and_formats):
    """Convert the date formats in specified columns in a DataFrame.
    Args:
        dataframe: The DataFrame containing the date columns.
        columns_and_formats: A dictionary where keys are column names and values are (input_format, output_format) tuples.
    Returns:
        dateframe: formatted dataframe
    """
    for column, (input_format, output_format) in columns_and_formats.items():
        # Ensure the specified column exists in the DataFrame
        if column not in dataframe.columns:
            print(f"Column '{column}' not found in the DataFrame.")
            continue

        # Convert the specified column to datetime using the input format
        try:
            dataframe[column] = pd.to_datetime(dataframe[column], format=input_format)
        except ValueError:
            print(f"Unable to convert column '{column}' to datetime using the input format '{input_format}'.")
            continue

        # Convert the datetime column to the desired output format
        dataframe[column] = dataframe[column].dt.strftime(output_format)

    return dataframe

In [4]:
# Define the columns and their format conversions
columns_and_formats = {
    'Start Date': ('%m/%d/%Y', '%d/%m/%Y'),
    'End Date': ('%m/%d/%Y', '%d/%m/%Y')
}

We have 6 files that starts with `MD`.  
We change it to `DM` format using the above function.  

In [5]:
# Loop through all "MD_" files
for csv_file in md_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv("../data/"+csv_file)
    # Extract the original text (file name without extension)
    original_text = csv_file.split('MD_')[1]
    # Create a new DataFrame name by prefixing 'DM_'
    new_df_name = f'DM_{original_text}'
    # Change the datetime format using the function convert_date_formats()
    new_df = convert_date_formats(df, columns_and_formats)
    # Save the DataFrame as a new CSV file
    if new_df_name.endswith(".csv"):  
        new_csv_file = f'{new_df_name}'
    else:
        new_csv_file = f'{new_df_name}.csv'
    new_df.to_csv("../data/"+new_csv_file, index=False)

Also some of the files have user id labelled as `uid`, while majority of them are labeled as `user_id`.   
Seems that some intern did the work, or maybe the old employee left.   
Let us solve the problem one by one.  
Anyways, we need to keep things constant.  

In [6]:
# Define anew location to save my changed files
save_file_path = "../pre_ana_data/"

In [7]:
# Get the DM files again to change uid.  
dm_files

['DM_2018 EVCP use Q3 and Q4.csv',
 'DM_2017 EVCP use Q3 and Q4.csv',
 'DM_2021 EVCP use Q1.csv',
 'DM_2014 EVCP use Q2.csv',
 'DM_2020 EVCP use Q3.csv',
 'DM_2015 EVCP use Q3 and Q4.csv',
 'DM_2015 EVCP use Q1 and Q2.csv',
 'DM_2016 EVCP use Q3 and Q4.csv',
 'DM_2020 EVCP use Q1 and Q2.csv',
 'DM_2019 EVCP use Q3 and Q4.csv',
 'DM_2019 EVCP use Q1 and Q2.csv',
 'DM_2018 EVCP use Q1 and Q2.csv',
 'DM_2016 EVCP use Q1 and Q2.csv',
 'DM_2014 EVCP use Q3 and Q4.csv',
 'DM_2017 EVCP use Q1 and Q2.csv']

In [8]:
# Regular expression to extract the year and quarter
regex_pattern = r'DM_(\d{4}).*?Q(\d)'

# Loop through each CSV file, read it into a DataFrame, and process the "User ID" or "Unique User ID" column
for csv_file in dm_files:
    # Extract the year and quarter from the file name using regex
    match = re.search(regex_pattern, csv_file)
    if match:
        year = match.group(1)
        quarter = match.group(2)
    else:
        print(match)
        year = "Unknown"
        quarter = "Unknown"
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Check if the DataFrame has a column named "Unique User ID"
    if 'Unique User ID' in df.columns:
        # Rename the "Unique User ID" column to "User ID"
        df = df.rename(columns={'Unique User ID': 'User ID'})
    
    # Process the "User ID" column
    df['User ID'] = df['User ID'].apply(lambda user_id: f"{year[-2:]}Q{quarter}{user_id}")
    
    # Save the updated DataFrame as a new CSV file
    updated_csv_file = f'Updated_{csv_file}'  # Add a prefix to the original file name
    df.to_csv(save_file_path+updated_csv_file, index=False)

In [9]:
# Read in all .csv files in data
path = "../pre_ana_data/"
os.chdir(path)
csv_files = [f for f in os.listdir() if f.endswith('.csv') if f != "updated_combined_ev_charging_data.csv"]

dfs = []
# Import all .csv files
for csv in csv_files:
    df = pd.read_csv(csv)
    dfs.append(df)

# Combine the dataframes
ev_df = pd.concat(dfs, ignore_index=True)

# Examine contents of dataframe
ev_df.head()

,Charging event,User ID,CP ID,Connector,Start Date,Start Time,End Date,End Time,Total kWh,Site,Model,Unnamed: 11,Unnamed: 12,Unnamed: 13,Cost
0,383322,14Q3User 31,70206,2,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,NaN,NaN,NaN
1,383031,14Q3User 635,70206,1,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,NaN,NaN,NaN
2,381777,14Q3User 38,70201,2,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,NaN,NaN,NaN
3,380951,14Q3User 629,70208,1,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,NaN,NaN,NaN
4,377470,14Q3User 626,70204,2,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,NaN,NaN,NaN


In [10]:
ev_df.shape

(24224, 15)

In [11]:
ev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24224 entries, 0 to 24223
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Charging event  24224 non-null  int64  
 1   User ID         24224 non-null  object 
 2   CP ID           24224 non-null  int64  
 3   Connector       24224 non-null  int64  
 4   Start Date      24224 non-null  object 
 5   Start Time      24224 non-null  object 
 6   End Date        23927 non-null  object 
 7   End Time        23927 non-null  object 
 8   Total kWh       23927 non-null  float64
 9   Site            24224 non-null  object 
 10  Model           24224 non-null  object 
 11  Unnamed: 11     0 non-null      float64
 12  Unnamed: 12     0 non-null      float64
 13  Unnamed: 13     0 non-null      float64
 14  Cost            2278 non-null   float64
dtypes: float64(5), int64(3), object(7)
memory usage: 2.8+ MB


As can be seen, there's no data in some columns.  
They should be removed completely.  
There are about 1k `Unique User ID` missing, and 2k `Cost` missing.  
They ammount to about 4% and 9.5% of the data!   
This is a significant ammount.  
It is advisable to take take a look and see if these missing data can be salvaged, or should they be dropped directly.  

In [12]:
# Change the name and drop columns that's filled with NA
col_keep = {"Charging event":"charging_event_id", 
            "User ID":"user_id", 
            "CP ID":"cp_id", 
            "Connector":"con_num", 
            "Start Date":"start_date", 
            "Start Time":"start_time", 
            "End Date":"end_date", 
            "End Time": "end_time",
            "Total kWh":"total_kwh", 
            "Site":"site", 
            "Model":"charger_model", 
            "Cost":"charging_cost"}
ev_df = ev_df.loc[:,[key for key in col_keep]].rename(columns=col_keep).copy(deep=True)
ev_df.to_csv("../pre_ana_data/updated_combined_ev_charging_data.csv", index=False)
ev_df.head()

,charging_event_id,user_id,cp_id,con_num,start_date,start_time,end_date,end_time,total_kwh,site,charger_model,charging_cost
0,383322,14Q3User 31,70206,2,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
1,383031,14Q3User 635,70206,1,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
2,381777,14Q3User 38,70201,2,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN
3,380951,14Q3User 629,70208,1,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
4,377470,14Q3User 626,70204,2,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN


In [13]:
ev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24224 entries, 0 to 24223
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   charging_event_id  24224 non-null  int64  
 1   user_id            24224 non-null  object 
 2   cp_id              24224 non-null  int64  
 3   con_num            24224 non-null  int64  
 4   start_date         24224 non-null  object 
 5   start_time         24224 non-null  object 
 6   end_date           23927 non-null  object 
 7   end_time           23927 non-null  object 
 8   total_kwh          23927 non-null  float64
 9   site               24224 non-null  object 
 10  charger_model      24224 non-null  object 
 11  charging_cost      2278 non-null   float64
dtypes: float64(2), int64(3), object(7)
memory usage: 2.2+ MB


In [14]:
unique_user = ev_df["user_id"].unique()

In [15]:
len(unique_user)

1897

This is the part where the string need to be converted to datetime format.  
This is so that the most important feature, the time of stay for the EV is calculated.  

In [16]:
ev_df = pd.read_csv("../pre_ana_data/updated_combined_ev_charging_data.csv")

In [17]:
ev_df.head()

,charging_event_id,user_id,cp_id,con_num,start_date,start_time,end_date,end_time,total_kwh,site,charger_model,charging_cost
0,383322.0,14Q3User 31,70206.0,2.0,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
1,383031.0,14Q3User 635,70206.0,1.0,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
2,381777.0,14Q3User 38,70201.0,2.0,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN
3,380951.0,14Q3User 629,70208.0,1.0,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN
4,377470.0,14Q3User 626,70204.0,2.0,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN


Convert the dates to date time.

In [18]:
# Combine "date" and "time" columns and convert to datetime
ev_df["start_datetime"] = pd.to_datetime(ev_df["start_date"] + ' ' + ev_df["start_time"], format='%d/%m/%Y %H:%M')

In [19]:
ev_df["end_datetime"] = pd.to_datetime(ev_df["end_date"] + ' ' + ev_df["end_time"], format='%d/%m/%Y %H:%M')

In [20]:
ev_df.head()

,charging_event_id,user_id,cp_id,con_num,start_date,start_time,end_date,end_time,total_kwh,site,charger_model,charging_cost,start_datetime,end_datetime
0,383322.0,14Q3User 31,70206.0,2.0,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-31 11:25:00,2014-12-31 11:35:00
1,383031.0,14Q3User 635,70206.0,1.0,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-30 13:58:00,2014-12-30 15:24:00
2,381777.0,14Q3User 38,70201.0,2.0,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-27 18:13:00,2014-12-27 19:08:00
3,380951.0,14Q3User 629,70208.0,1.0,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-24 11:37:00,2014-12-24 12:06:00
4,377470.0,14Q3User 626,70204.0,2.0,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-18 09:29:00,2014-12-18 11:56:00


In [21]:
# Calculate the duration and save it as a new column
ev_df["duration_hr"] = (ev_df["end_datetime"] - ev_df["start_datetime"]).dt.total_seconds() / 3600

In [22]:
# Get the day of the week for start_datetime and end_datetime
ev_df["start_day"] = ev_df["start_datetime"].dt.day_name()
ev_df["end_day"] = ev_df["end_datetime"].dt.day_name()

In [23]:
ev_df.head()

,charging_event_id,user_id,cp_id,con_num,start_date,start_time,end_date,end_time,total_kwh,site,charger_model,charging_cost,start_datetime,end_datetime,duration_hr,start_day,end_day
0,383322.0,14Q3User 31,70206.0,2.0,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-31 11:25:00,2014-12-31 11:35:00,0.166667,Wednesday,Wednesday
1,383031.0,14Q3User 635,70206.0,1.0,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-30 13:58:00,2014-12-30 15:24:00,1.433333,Tuesday,Tuesday
2,381777.0,14Q3User 38,70201.0,2.0,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-27 18:13:00,2014-12-27 19:08:00,0.916667,Saturday,Saturday
3,380951.0,14Q3User 629,70208.0,1.0,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-24 11:37:00,2014-12-24 12:06:00,0.483333,Wednesday,Wednesday
4,377470.0,14Q3User 626,70204.0,2.0,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-18 09:29:00,2014-12-18 11:56:00,2.450000,Thursday,Thursday


In [24]:
def categorize_time(time_str):
    if pd.isna(time_str) or not isinstance(time_str, str):
        return "Invalid or Missing Time"

    try:
        # Convert the time string to a datetime object
        time_obj = dt.datetime.strptime(time_str, '%H:%M')
        # Get the hour component from the datetime object
        hour = time_obj.hour

        # Define the time categories
        categories = {
            'early morning': range(0, 6),
            'morning': range(6, 12),
            'afternoon': range(12, 18),
            'evening': range(18, 24)
        }

        # Determine the category for the given time
        time_category = None
        for category, hours in categories.items():
            if hour in hours:
                time_category = category
                break

        return time_category
    except ValueError:
        return "Invalid or Missing Time"

In [25]:
ev_df["start_period"] = ev_df["start_time"].apply(categorize_time)

In [26]:
ev_df.head()

,charging_event_id,user_id,cp_id,con_num,start_date,start_time,end_date,end_time,total_kwh,site,charger_model,charging_cost,start_datetime,end_datetime,duration_hr,start_day,end_day,start_period
0,383322.0,14Q3User 31,70206.0,2.0,31/12/2014,11:25,31/12/2014,11:35,0.57,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-31 11:25:00,2014-12-31 11:35:00,0.166667,Wednesday,Wednesday,morning
1,383031.0,14Q3User 635,70206.0,1.0,30/12/2014,13:58,30/12/2014,15:24,4.52,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-30 13:58:00,2014-12-30 15:24:00,1.433333,Tuesday,Tuesday,afternoon
2,381777.0,14Q3User 38,70201.0,2.0,27/12/2014,18:13,27/12/2014,19:08,0.00,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-27 18:13:00,2014-12-27 19:08:00,0.916667,Saturday,Saturday,evening
3,380951.0,14Q3User 629,70208.0,1.0,24/12/2014,11:37,24/12/2014,12:06,3.16,Elland Road Park and Ride,APT 7kW Dual Outlet,NaN,2014-12-24 11:37:00,2014-12-24 12:06:00,0.483333,Wednesday,Wednesday,morning
4,377470.0,14Q3User 626,70204.0,2.0,18/12/2014,09:29,18/12/2014,11:56,3.16,Woodhouse Lane Car Park,APT 7kW Dual Outlet,NaN,2014-12-18 09:29:00,2014-12-18 11:56:00,2.450000,Thursday,Thursday,morning


In [29]:
ev_df.shape

(121120, 18)

In [22]:
ev_df.to_csv("../pre_ana_data/updated_combined_ev_charging_data.csv", index=False)